In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

## 1. Data Discretization

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, run_job, test_bins
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. Complete Data Loading

First of all we need to loading the complete data into the quantum state:

$$|\Psi\rangle=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

where $|\Psi\rangle$ is the notation for a n+1 qbit quantum state

From module **QuantumMultiplexors_Module_examples** fucntion **LoadingData** can be used. This function receives numpy arrays with probability and function and outputs the quantum program for loading complete data and the mandatory $\mathcal{P}$ y $\mathcal{R}$ gates to perform the loading.

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data

In [ ]:
qLprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)

In [ ]:
%qatdisplay P_Gate
%qatdisplay R_gate

circuit = qLprog.to_circ(submatrices_only=True)
%qatdisplay circuit

In [ ]:
job = circuit.to_job(qubits=[n_qbits])
result = run_job(lineal_qpu.submit(job))
InitialState = postprocess_results(result)
#Check
print(np.isclose(InitialState['Probability'][1], sum(p_X*f_X)))

# 3. Amplitude Amplification

The quantum state resulting from the loading proccess $|\Psi\rangle$ can be expressed as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$ (each one is formed by n qbits):

$$|\Psi\rangle=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Where $|\Psi_{0}\rangle$ and $|\Psi_{1}\rangle$ are the following orthonormal states:

$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|0\rangle$$
$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|1\rangle$$


Where $|x\rangle^{n}$ is a notation for a quantum state of n qbits.


The idea behind Quantum Phase Amplification is increase the probability of the $|\Psi_{1}\rangle$  using a strategy based on the Groover Algorithm. 

An Operator $\hat{Q}$ should be applied an optimal number of times $k$ in order to maximize probability of measure $|\Psi_{1}\rangle$. This operator is: 

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Where $\hat{U}_{|\Psi\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ are:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$

Module **PhaseAmplification_Module** have all the functions in order to create properly gates for this operators

### 3.1 Operador $\hat{U}_{|\Psi_{0}\rangle}$

First mandatory operator is:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

Equivalent circuit for the operator is:

$$\hat{U}_{|\Psi_{0}\rangle }=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

Apply this operator on state $|\Psi\rangle$:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

So operator $\hat{U}_{|\Psi_{0}\rangle }$ do a reflection operation around the axis defined by the state $|\Psi_{1}\rangle$


Abstract Gate **UPhi0_Gate** from **PhaseAmplification_Module** allow us do the operation:

In [ ]:
qLprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
#Initial State
circuit = qLprog.to_circ(submatrices_only=True)
job = circuit.to_job()
result = run_job(lineal_qpu.submit(job))
InitialState = postprocess_results(result)
#Applied Gate
from PhaseAmplification_Module import uphi0_gate
UPhi0_Gate = uphi0_gate(qLprog.qbit_count)
qLprog.apply(UPhi0_Gate, qLprog.registers)

%qatdisplay UPhi0_Gate --depth 1
circuit = qLprog.to_circ(submatrices_only=True)
%qatdisplay circuit

job = circuit.to_job()
result = run_job(lineal_qpu.submit(job))
UPhi0_State = postprocess_results(result)

For testing the operator we need to check that: 
* Sates with final qbit $|0\rangle$ ($|\Psi_{0}\rangle$) change the sign with respect to the initial state $|\Psi\rangle$ 
* States with final qbit in $|1\rangle$ ($|\Psi_{1}\rangle$) are the same that in the initial state $|\Psi\rangle$

In [ ]:
#Testing Final qbit |0> should be of different sign
LastQbit0 = np.isclose(
    InitialState.loc[[i for i in range(0, len(InitialState), 2)]]['Amplitude'], 
    -UPhi0_State.loc[[i for i in range(0, len(InitialState), 2)]]['Amplitude']
).all()
#Testing Final qbit |1> should be of same sign
LastQbit1 = np.isclose(
    InitialState.loc[[i for i in range(1, len(InitialState), 2)]]['Amplitude'], 
    UPhi0_State.loc[[i for i in range(1, len(InitialState), 2)]]['Amplitude']
).all()

print(LastQbit0 and LastQbit1)

### 3.2 Operador $\hat{U}_{|\Psi\rangle}$

Operator $\hat{U}_{|\Psi\rangle}$ is based in Groover's difusor:

$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$

Additionally we know that:

$$|\Psi\rangle=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle^{\otimes n}\otimes|0\rangle$$

Then the operator can be decomposed in the following way:

$$\hat{U}_{|\Psi\rangle } =\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)\hat{D}_{0} \left(\hat{P_{n}}\otimes I\right)^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Where $\hat{D}_{0}$ is a reflection around the perpendicular state to $|0^{n+1}\rangle$

$$\hat{D}_{0} = \hat{I}-2|0^{n+1}\rangle\langle0^{n+1}|$$

In this case $|0^{n+1}\rangle$ is the zero state for n+1 qbits

#### 3.2.1 Implementación  $\hat{D}_{0}$

The equivalent circuit for $\hat{D}_{0}$ is:

$$\hat{D}_{0} = \hat{I}-2|0^{n+1}\rangle\langle0^{n+1}|= \hat{X}^{\otimes n+1} c^{n}Z \hat{X}^{\otimes n+1}$$

The *AbstractGate* **D0_Gate** from **PhaseAmplification_Module** allow us create the operator:

In [ ]:
qLprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
#Initial State
circuit = qLprog.to_circ(submatrices_only=True)
job = circuit.to_job()
result = run_job(lineal_qpu.submit(job))
InitialState = postprocess_results(result)
#Applied Gate
from PhaseAmplification_Module import d0_gate
D0 = d0_gate(qLprog.qbit_count)
qLprog.apply(D0, qLprog.registers)

%qatdisplay D0 --depth 1
circuit = qLprog.to_circ(submatrices_only=True)
%qatdisplay circuit

job = circuit.to_job()
result = run_job(lineal_qpu.submit(job))
D0_State = postprocess_results(result)

Operator $\hat{D}_{0}$ over state  $|\Psi\rangle$:

$$\hat{D}_{0}|\Psi\rangle = \hat{I}|\Psi\rangle-2|0^{n+1}\rangle\langle0^{n+1}|\Psi\rangle^{n+1}$$


It can be demostrated that the only difference between $\hat{D}_{0}|\Psi\rangle$ and $|\Psi\rangle$ is that the component $|0^{n+1}\rangle$ the sign changes!!

In [ ]:
#Testing
C01 = np.isclose(InitialState['Amplitude'].loc[0], -D0_State['Amplitude'].loc[0])
C02 = np.isclose(InitialState['Amplitude'].loc[1:], D0_State['Amplitude'].loc[1:]).all()
print(C01 and C02)

#### 3.2.2 Implementation  $\hat{U}_{|\Psi}\rangle$

Finally the function **Load_UPhi_Gate** from **PhaseAmplification_Module** creates a customized AbstractGate that implements $\hat{U}_{|\Psi\rangle }$. This functions needs the initial $\mathcal{P}$ y $\mathcal{R}$ gates used for loading the data.

This operator is reflection around the ortoghonal state to $|\Psi\rangle$

In [ ]:
qLprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
#Initial State
circuit = qLprog.to_circ(submatrices_only=True)
job = circuit.to_job()
result = run_job(lineal_qpu.submit(job))
InitialState = postprocess_results(result)
#Applied Gate
from PhaseAmplification_Module import load_uphi_gate
UPhi = load_uphi_gate(P_Gate, R_gate)
qLprog.apply(UPhi, qLprog.registers)

%qatdisplay UPhi --depth 1
circuit = qLprog.to_circ(submatrices_only=True)
%qatdisplay circuit

job = circuit.to_job()
result = run_job(lineal_qpu.submit(job))
UPhi_State = postprocess_results(result)

In this case 
$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = \hat{I}|\Psi\rangle - 2|\Psi\rangle \langle \Psi|\Psi\rangle$$

$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = |\Psi\rangle - 2|\Psi\rangle = -|\Psi\rangle$$



In [ ]:
#Testing
np.isclose(UPhi_State['Amplitude'], - InitialState['Amplitude']).all()

### 3.3 Operador $\hat{Q}$

Finally we can implement the desired Operator $\hat{Q}$ 

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

This operator is created using the function **Load_Q_Gate** from **PhaseAmplification_Module**



In [ ]:
#Complete Circuit
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
from PhaseAmplification_Module import load_q_gate
Q_Gate = load_q_gate(P_Gate, R_gate)
qbits = Qprog.registers
Qprog.apply(Q_Gate, qbits)
circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])
result = run_job(lineal_qpu.submit(job))
QPhi_State = postprocess_results(result)

In [ ]:
%qatdisplay circuit

To test if operator was implemented properly we know that the quantum state resulting from the complete loading proccess $|\Psi\rangle_{n+1}$ can be expressed as a linear combination of twor othogonal states $|\Psi_{1}\rangle$ and $|\Psi_{0}\rangle$:

$$|\Psi\rangle=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

where:

$$a = \sum_{x=0}^{2^{n}-1} p(x)f(x)$$

Making the following identities:

* $\sqrt{a} = \sin{\theta}$
* $\sqrt{1-a} = \cos{\theta}$
* $ a = \sum_{n=0}^{2^{n}-1} p(x)f(x) = \sin^{2}{\theta}$

Then:

$$|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$$

So using the initial state we are going to calculate $\theta$.

In [ ]:
#initial Loaded State
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
#Get initial State
circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])
result = run_job(lineal_qpu.submit(job))
Phi = postprocess_results(result)

#Get Q*Phi state
from PhaseAmplification_Module import load_q_gate
Q_Gate = load_q_gate(P_Gate, R_gate)

qbits = Qprog.registers
Qprog.apply(Q_Gate, qbits)
circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])
result = run_job(lineal_qpu.submit(job))
Q_Phi = postprocess_results(result)

In [ ]:
Phi

In [ ]:
Q_Phi


The operator $\hat{Q}$ can be viewed as a Y-Rotation of $\theta$ on $|\Psi\rangle$: 

$$\hat{Q}|\Psi\rangle= \hat{R}_{y}(2*\theta)|\Psi\rangle=\hat{R}_{y}(2*\theta)(\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle) = \sin{3\theta}\Psi_{1}\rangle+\cos{3\theta}|\Psi_{0}\rangle$$

So starting from $|\Psi\rangle$ we can obtain the angle $\theta$

In [ ]:
#Calculating Theta using the quantum state from loading data: LResults
def GetAngle(Array):
    Modulo = np.linalg.norm(Array)
    cosTheta = Array[0]/Modulo
    Theta0 = np.arccos(cosTheta)
    sinTheta = Array[1]/Modulo
    Theta1 = np.arcsin(sinTheta)
    #print(Modulo, cosTheta, sinTheta, Theta0, Theta1)
    return Theta0

#First get the Amplitudes for Phi state
a0 = np.sqrt(Phi.iloc[0]['Probability'])
a1 = np.sqrt(Phi.iloc[1]['Probability'])
#Quantum state |Phi>_{n+1}
Psi_ = np.array([a0, a1])
theta = GetAngle(Psi_)
print('theta: {}'.format(theta))
print('Psi_: {}'.format(Psi_))

Now we have $\theta$ and the $|\Psi\rangle$

In [ ]:
#In order to test that this theta is ok we can compute it from p(x) and f(x)
a = sum(p_X*f_X)
sqrt_a = np.sqrt(a)
theta2 = np.arcsin(sqrt_a)
print('theta2: {}'.format(theta2))

print('Is theta equals to theta2: {}'.format(np.isclose(theta, theta2)))

Now we apply Rotation of angle $\theta$ to state $|\Psi\rangle$ (Psi_):

In [ ]:
#Create a Rotation of 2*theta
c, s = np.cos(2*theta), np.sin(2.*theta)
R = np.array(((c, -s), (s, c)))
#Apply Ry(2*theta) to quantum state |Phddi>_{n+1}
RotationResults = np.dot(R, Psi_)

So in this case the Rotated vector should be the state correspondient to the operation: $\hat{Q}|\Psi\rangle$. In our case we have the probabilities for $\hat{Q}|\Psi\rangle$ so we can square the Rotated Vector and compared with the probabilities.

This is done in the following cells:


In [ ]:
print('Square Of Rotated Vector: {}'.format(RotationResults**2))
print('Probabilities for QPhi_State: {} '.format(list(QPhi_State['Probability'])))
print('Square of the RotateState equal to Probabilities of Q|Phi> state : {}'.format(
    np.isclose(RotationResults**2, QPhi_State['Probability']).all())
     )

As can be seen the application of a Rotation of $2*\theta$ on $|\Psi\rangle$ give us the same proability configuration that apply a $\hat{Q}|\Psi\rangle$. 